# EVRP Algorithm
___


### Index

___


In [1]:
import sys
import os
from pathlib import Path 
sys.path.insert(1 ,os.path.dirname(Path(os.path.abspath("__file__")).resolve().parent))

import math
import folium
import pickle
import numpy as np
import pandas as pd
from geopy import Point, Nominatim
from geopy.distance import geodesic
import itertools 

import pyomo.solvers
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from datetime import datetime

In [2]:
# initial environ settings 
file_path = "../data/input/EVRP - Template.xlsx"
model = pyo.ConcreteModel()
opt = SolverFactory('glpk')

In [3]:
# Constantes
velocidade_media = 20
tempo_serviço = 0.5

In [4]:
# Cálculo de distância entre pontos i e j
def calcular_distancia(latitude_i, longitude_i, latitude_j, longitude_j):
    return math.ceil(geodesic(Point(latitude=latitude_i, longitude=longitude_i), Point(latitude=latitude_j, longitude=longitude_j)).km)

In [5]:
# CONJUNTOS
# Clientes
df_clientes = pd.read_excel(file_path, sheet_name='Clientes')
clientes = df_clientes.to_dict("records")
clientes = {
    cliente['Cliente']: {
        'Latitude': cliente['Latitude'],
        'Longitude': cliente['Longitude'],
        'Quantidade': cliente['Quantidade'],
        'Leadtime': cliente['Leadtime']
    }
    for cliente in clientes
}

# Pontos de Recarga
df_pontos_recarga = pd.read_csv("../data/input/charging_stations.csv", sep=";", decimal=".", encoding="utf-8")
pontos_recarga = df_pontos_recarga.to_dict("records")
pontos_recarga = {
    ponto['ID']: {
        'Latitude': ponto['Latitude'],
        'Longitude': ponto['Longitude'],
        'Potência de Recarga': ponto['Potência de Recarga']
    }
    for ponto in pontos_recarga
}

# Veículos
df_veiculos = pd.read_excel(file_path, sheet_name='Veículos')
veiculos = df_veiculos.to_dict("records")
veiculos = {
    veiculo['Veículo']: {
        'Capacidade da Bateria (kWh)': veiculo['Capacidade da Bateria (kWh)'],
        'Consumo (kWh/km)': veiculo['Consumo (kWh/km)'],
        'Autonomia (km)': veiculo['Capacidade da Bateria (kWh)'] / veiculo['Consumo (kWh/km)']
    }
    for veiculo in veiculos
}

# Centros de Distribuição
df = pd.read_excel(file_path, sheet_name='Centro de Distribuição')
centros_distribuicao = df.to_dict("records")
centros_distribuicao = {
    centro['Centro de Distribuição']: {
        'Latitude': centro['Latitude'],
        'Longitude': centro['Longitude']
    }
    for centro in centros_distribuicao
}

# Todos os pontos
pontos = {}
pontos.update(clientes)
pontos.update(pontos_recarga)
pontos.update(centros_distribuicao)

In [6]:
# Sets
model.C = pyo.Set(initialize=clientes.keys(), doc='Clientes')
model.R = pyo.Set(initialize=pontos_recarga.keys(), doc='Pontos de Recarga')
#model.P = pyo.Set(initialize=pedidos.keys(), doc='Pedidos')
model.K = pyo.Set(initialize=veiculos.keys(), doc='Veículos')
model.zero = pyo.Set(initialize=centros_distribuicao.keys(), doc='Centros de Distribuição')

model.N = pyo.Set(initialize=model.C.union(model.R), doc='Conjunto de pontos - Clientes e Pontos de Recarga')
model.Nlinha = pyo.Set(initialize=model.N.union(model.zero), doc='Conjuntos de pontos - N e Centros de Distribuição')

In [7]:
# Parametros
def atribuir_distancia(model, i, j):
    try:
        return calcular_distancia(pontos[i]['Latitude'], pontos[i]['Longitude'], pontos[j]['Latitude'], pontos[j]['Longitude'])
    except KeyError:
        return 0
model.d = pyo.Param(model.Nlinha, model.Nlinha, initialize=atribuir_distancia, doc='Distância entre pontos i e j (km)')

#def atribuir_capacidade(model, k):
#    try:
#        return veiculos[k]['Capacidade da Bateria (kWh)']
#    except KeyError:
#        return 0
#model.Q = pyo.Param(model.K, within=pyo.NonNegativeIntegers, initialize=atribuir_capacidade, doc='Capacidade da bateria do veículo (kWh)')

model.v = pyo.Param(within=pyo.NonNegativeIntegers, initialize=velocidade_media, doc='Velocidade média dos Veículos (km/h)')

def atribuir_autonomia(model, k):
    try:
        return veiculos[k]['Autonomia (km)']
    except KeyError:
        return 0
model.a = pyo.Param(model.K, within=pyo.NonNegativeReals, initialize=atribuir_autonomia, doc='Autonomia do veículo (km)')

def atribuir_consumo(model, k):
    try:
        return veiculos[k]['Consumo (kWh/km)']
    except KeyError:
        return 0
model.c = pyo.Param(model.K, within=pyo.NonNegativeReals, initialize=atribuir_consumo, doc='Consumo por Km do veículo (kWh/km)')

def atribuir_potencia_recarga(model, r):
    try:
        return pontos_recarga[r]['Potência de Recarga']
    except KeyError:
        return 0
model.r = pyo.Param(model.Nlinha, within=pyo.NonNegativeReals, initialize=atribuir_potencia_recarga, doc='Potência de recarga do ponto de recarga (kWh)')

#def atribuir_demanda(model, i):
#    try:
#        return clientes[i]['Quantidade']
#    except KeyError:
#        return 0
#model.q = pyo.Param(model.C, within=pyo.NonNegativeReals, initialize=atribuir_demanda, doc='Demanda do pedido do cliente i (unidades)')

def atribuir_leadtime(model, i):
    try:
        return clientes[i]['Leadtime']
    except KeyError:
        return 0
model.l = pyo.Param(model.C, within=pyo.NonNegativeReals, initialize=atribuir_leadtime, doc='Leadtime do pedido do cliente i (horas)')

model.s = pyo.Param(within=pyo.NonNegativeReals, initialize=tempo_serviço, doc='Tempo de serviço (horas)')

model.tzero = pyo.Param(within=pyo.NonNegativeReals, initialize=0, doc='Tempo inicial (horas)')

In [8]:
# Variáveis de decisão
model.x = pyo.Var(model.K, model.Nlinha, model.Nlinha, within=pyo.Binary, initialize=0, doc='Variável de decisão que indica se o veículo 𝑘 vai de i para j')
def y_init(model, k, i):
    if i in model.zero:
        return model.a[k]
    return 0
model.y = pyo.Var(model.K, model.Nlinha, within=pyo.NonNegativeIntegers, initialize=y_init, doc='Quantidade de bateria do veículo 𝑘 no ponto i')
model.t = pyo.Var(model.K, model.Nlinha, within=pyo.NonNegativeReals, initialize=0, doc='Variável de decisão que indica o tempo de chegada do veículo 𝑘 no ponto i')
model.u = pyo.Var(model.K, model.Nlinha, within=pyo.NonNegativeIntegers, doc='Variável de decisão que indica a carga adicionada ao veículo 𝑘 no ponto i')
#model.z = pyo.Var(model.K, model.R, within=pyo.Binary, doc='Variável binária que indica se o veículo recarrega no ponto i')

In [9]:
# Função Objetivo
def FuncaoObj(model):
    return sum(model.x[k, i, j] * model.d[i, j] for k in model.K for i in model.Nlinha for j in model.Nlinha)
model.obj = pyo.Objective(rule=FuncaoObj, sense=pyo.minimize, doc='Função Objetivo para Otimização do Problema de Roteirização de Veículos Elétricos - EVRP')

In [10]:
# Restrições
def atendimento_pedido_rule(model, i):
    return sum(model.x[k, i, j] for k in model.K for j in model.Nlinha) >= 1
model.atendimento_cliente = pyo.Constraint(model.C, rule=atendimento_pedido_rule, doc='Restrição de atendimento ao cliente i')

def capacidade_bateria_rule(model, k, i):
    return model.y[k, i] <= model.a[k]
model.capacidade_bateria = pyo.Constraint(model.K, model.Nlinha, rule=capacidade_bateria_rule, doc='Restrição de capacidade da bateria do veículo k')

def carregamento_veiculo_rule(model, k, i):
    #return model.y[k, i] + (model.u[k, i] * model.z[k, i]) <= model.a[k]
    return model.y[k, i] + model.u[k, i] <= model.a[k]
model.carregamento_veiculo = pyo.Constraint(model.K, model.Nlinha, rule=carregamento_veiculo_rule, doc='Restrição de carregamento do veículo k')

def recarga_apenas_em_pontos_de_recarga_rule(model, k, i):
    if i not in model.R:
        #return model.z[k, i] == 0
        return model.u[k, i] == 0
    return pyo.Constraint.Skip
model.recarga_pontos_recarga = pyo.Constraint(model.K, model.Nlinha, rule=recarga_apenas_em_pontos_de_recarga_rule, doc='Restrição de recarga apenas em pontos de recarga')

def autonomia_rule(model, k, i, j):
    return model.y[k, i] - model.y[k, j] >= model.d[i, j] * model.x[k, i, j]
model.autonomia = pyo.Constraint(model.K, model.Nlinha, model.Nlinha, rule=autonomia_rule, doc='Restrição de autonomia do veículo k')

def leadtime_rule(model, k, i):
    return model.t[k, i] + model.s <= model.l[i]
model.leadtime = pyo.Constraint(model.K, model.C, rule=leadtime_rule, doc='Restrição de leadtime do pedido do cliente i')

def conservacao_fluxo_rule(model, k, i):
    return sum(model.x[k, i, j] for j in model.Nlinha) == sum(model.x[k, j, i] for j in model.Nlinha)
model.conservacao_fluxo = pyo.Constraint(model.K, model.Nlinha, rule=conservacao_fluxo_rule, doc='Restrição de conservação de fluxo')

def tempo_de_servico_rule(model, k, i, j):
    #return model.t[k, i] + model.s[p] + (model.d[i, j]/model.v) + (model.u[k, i] * model.z[k, i] * (model.r[i]/model.c[k])) <= model.t[k, j]
    return model.t[k, i] + model.s + (model.d[i, j]/model.v) + (model.u[k, i] * (model.r[i]/model.c[k])) <= model.t[k, j]
model.tempo_de_servico = pyo.Constraint(model.K, model.Nlinha, model.Nlinha, rule=tempo_de_servico_rule, doc='Restrição de tempo de serviço')

def partida_deposito_rule(model, k, zero):
    return sum(model.x[k, zero, j] for j in model.Nlinha) == 1
model.partida_deposito = pyo.Constraint(model.K, model.zero, rule=partida_deposito_rule, doc='Restrição de partida do depósito')

def retorno_deposito_rule(model, k, zero):
    return sum(model.x[k, i, zero] for i in model.Nlinha) == 1
model.retorno_deposito = pyo.Constraint(model.K, model.zero, rule=retorno_deposito_rule, doc='Restrição de retorno ao depósito')

In [11]:
model.y.display()

y : Quantidade de bateria do veículo 𝑘 no ponto i
    Size=19320, Index=K*Nlinha
    Key             : Lower : Value : Upper : Fixed : Stale : Domain
       ('V1', 'C1') :     0 :     0 :  None : False : False : NonNegativeIntegers
      ('V1', 'C10') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C100') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C101') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C102') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C103') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C104') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C105') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C106') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C107') :     0 :     0 :  None : False : False : NonNegativeIntegers
     ('V1', 'C109') :     0 : 

In [12]:
results = opt.solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 18747444
  Number of variables: 9383201
  Number of nonzeros: 74611600
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 117.39114284515381


In [13]:
for k, i, j in model.x:
    if model.x[k, i, j].value > 0:
        print(f"Veículo {k} vai de {i} para {j}")